In [29]:
#final value
import cv2
import numpy as np
from math import atan2, cos, sin, sqrt, pi
hi = 0
ha = 0
si = 0
sa = 0
vi = 0
va = 60

In [30]:
def drawAxis(img, p_, q_, color, scale):
    p = list(p_)
    q = list(q_)

    ## [visualization1]
    angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
    #hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
    hypotenuse = 100

    # Here we lengthen the arrow by a factor of scale
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA)

    # create the arrow hooks
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA)

    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA)
    ## [visualization1]


In [31]:
def getOrientation(pts, img):
    ## [pca]
    # Construct a buffer used by the pca analysis
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]
    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
    # Store the center of the object
    cntr = (int(mean[0,0]), int(mean[0,1]))
    ## [pca]
    ## [visualization]
    # Draw the principal components
    cv2.circle(img, cntr, 3, (255, 0, 255), 2)
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    drawAxis(img, cntr, p1, (255, 255, 0), 1) #Blue line
    #drawAxis(img, cntr, p2, (0, 0, 255), 5)  #Red line

    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
    ## [visualization]

    # Label with the rotation angle
    label = "  Rotation Angle: " + str(-int(np.rad2deg(angle)) - 90) + " degrees"
    textbox = cv2.rectangle(img, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1)
    cv2.putText(img, label, (cntr[0], cntr[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA) 
    cv2.imshow("img",img)
    cv2.waitKey(1)
    #cv2.destroyAllWindows()
    return angle


In [89]:
angle_value = np.ones(7)*1.377
a_t = 0
for i in range(0,3000):
    #a = "imgs/c"+str(i)+".png"
    a = "rgb_imgs/"+str(i)+".png"
    #print(a)
    img = cv2.imread(a)
    img1 = cv2.Canny(img,50,255)
    contours, hierarchy = cv2.findContours(img1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    c = []
    for count in contours:
        epsilon = 0.01 * cv2.arcLength(count, True)
        approximations = cv2.approxPolyDP(count, epsilon, True)
        if(cv2.contourArea(count)>800):
            c.append(count)
    cx = 0
    cy = 0
    if(len(c)>0):
        pt = c[0]
        for i in c:
            pt = np.append(pt,i,axis=0)
            cnt = i
            M = cv2.moments(cnt)
            cx += int(M['m10']/M['m00'])
            cy += int(M['m01']/M['m00'])
        cx = int(cx/len(c))
        cy = int(cy/len(c))
        rect = cv2.minAreaRect(pt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(img,[box],0,(0,0,255),2)
#         d1 = (box[3][1] - box[0][1])*(box[3][1] - box[0][1]) + (box[3][0] - box[0][0])*(box[3][0] - box[0][0])
#         d2 = (box[1][1] - box[0][1])*(box[1][1] - box[0][1]) + (box[1][0] - box[0][0])*(box[1][0] - box[0][0])
#         if(d1 > d2):
        angle = atan2(box[3][1] - box[0][1], box[3][0] - box[0][0])
        #else:
            #angle = atan2(box[1][1] - box[0][1], box[1][0] - box[0][0])
        a_t = (angle+a_t-angle_value[0])/7
        np.delete(angle_value,[0])
        angle_value = np.append(angle_value,[a_t],axis=0)
        angle = a_t
        hyp = 100
        sx = int(cx - hyp*cos(angle))
        sy = int(cy - hyp*sin(angle))
        cv2.line(img, (cx,cy), (sx,sy), (255,0,0), 3, cv2.LINE_AA)
        #########################################
        ############ROI OF CAR ##################
        #########################################
        
#         roi = np.zeros((480,640),np.int8)
#         cv2.fillPoly(roi, [box], 255)
#         img = cv2.bitwise_and(img,img,mask=roi)
        
#         lower = np.array([hi,si,vi])
#         upper = np.array([ha,sa,va])
#         imghsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
#         mask = cv2.inRange(imghsv,lower,upper)
#         mask = cv2.bitwise_and(mask,mask,mask=roi)
        #imgq = cv2.bitwise_and(,img,mask=roi)
        #kernel = np.ones((5,5), np.uint8)
        #mask = cv2.dilate(mask, kernel, iterations=10)
        #mask = cv2.erode(mask, kernel, iterations=5)
#         getOrientation(pt,img)
#         imgc = img.copy()
#         cv2.circle(imgc,(cx,cy),7,(0,255,0),-1)
        imgc = img.copy()
        #cv2.drawContours(imgc, [approximations], 0, (0,0,255), 3)
        cv2.imshow("img",img)
        cv2.waitKey(1)
        #cv2.destroyAllWindows()
cv2.destroyAllWindows()

In [68]:
angle_value = np.zeros(20)
a_t = 0

In [80]:
a_t = angle+a_t-angle_value[0]
np.delete(angle_value,[0])
angle_value = np.append(angle_value,[angle],axis=0)
print(angle_value,a_t)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.53854944 1.53854944 1.53854944 1.53854944
 1.53854944 1.53854944 1.53854944] 18.46259333231571
